In [ ]:
from keras.datasets import imdb 
from keras.preprocessing import sequence
import keras 
import tensorflow as tf 
import os 
import numpy as np

VOCAB_SIZE = 88584
MAXLEN = 250
BATCH_SIZE = 64 

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

In [ ]:
train_data[0]

In [ ]:
len(train_data[0])
len(train_data[1]) 

In [ ]:
#more_preprocessing changing the size of all the sentences to MAXLEN=250
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)
train_data[0]

In [ ]:
model = tf.keras.Sequential([
      tf.keras.layers.Embedding(VOCAB_SIZE, 32),
      tf.keras.layers.LSTM(32),
      tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

In [ ]:
model.compile(
   optimizer = 'rmsprop',
   loss = 'binary_crossentropy',
   metrics = ['accuracy']
)

In [ ]:
history = model.fit(train_data, train_labels, validation_split=0.2, epochs=2)

In [ ]:
result = model.evaluate(test_data, test_labels)
print(result)

In [ ]:
#lets add a function to encode the words 
word_index = imdb.get_word_index()

def encode_text(text):
    tokens = keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

In [ ]:
#lets make a decode function also 
reverse_word_index = {value: key for (key,value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
        if num!=PAD:
            text = text + reverse_word_index[num] + " "
    return text[:-1]
print(decode_integers(encoded))

In [ ]:
#time for prediction 

def predict(text):
    encoded_text = encode_text(text)
    pred = np.zeros((1,250))
    pred[0] = encoded_text
    result = model.predict(pred)
    return result

text = str(input("Write a short and crisp movie review ? \n"))
if predict(text)>0.45 and predict(text)<0.55:
        print("\nYour Review suggest that you have mixed feelings")
elif predict(text)>=0.55: 
        print("\nYour Review tells that you liked the movie")    
elif predict(text)<=0.45: 
        print("\nYour Review tells that you didn't liked the movie")    
predict(text)